In [68]:
#Import Libraries
import numpy as np
import pandas as pd
import pandas_ta as ta
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn import datasets
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score

In [41]:
df = pd.read_csv('sp500.csv')
df = df.drop('Unnamed: 0', 1)
df

/tmp/ipykernel_12536/3620037511.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('Unnamed: 0', 1)


,Index,GSPC.Open,GSPC.High,GSPC.Low,GSPC.Close,GSPC.Volume,GSPC.Adjusted
0,1970-01-02,92.06,93.54,91.79,93.00,8050000,93.00
1,1970-01-05,93.00,94.25,92.53,93.46,11490000,93.46
2,1970-01-06,93.46,93.81,92.13,92.82,11460000,92.82
3,1970-01-07,92.82,93.38,91.93,92.63,10010000,92.63
4,1970-01-08,92.63,93.47,91.99,92.68,10670000,92.68
...,...,...,...,...,...,...,...
10017,2009-09-09,1025.36,1036.34,1023.97,1033.37,5202550000,1033.37
10018,2009-09-10,1032.99,1044.14,1028.04,1044.14,5191380000,1044.14
10019,2009-09-11,1043.92,1048.18,1038.40,1042.73,4922600000,1042.73
10020,2009-09-14,1040.15,1049.74,1035.00,1049.34,4979610000,1049.34


In [42]:
#Number of missing values in the dataset
df.isna().sum()

Index            0
GSPC.Open        0
GSPC.High        0
GSPC.Low         0
GSPC.Close       0
GSPC.Volume      0
GSPC.Adjusted    0
dtype: int64

In [43]:
df['Daily_average_price'] = (df['GSPC.High']+df['GSPC.Low']+df['GSPC.Close'])/3
df.head()

,Index,GSPC.Open,GSPC.High,GSPC.Low,GSPC.Close,GSPC.Volume,GSPC.Adjusted,Daily_average_price
0,1970-01-02,92.06,93.54,91.79,93.00,8050000,93.00,92.776667
1,1970-01-05,93.00,94.25,92.53,93.46,11490000,93.46,93.413333
2,1970-01-06,93.46,93.81,92.13,92.82,11460000,92.82,92.920000
3,1970-01-07,92.82,93.38,91.93,92.63,10010000,92.63,92.646667
4,1970-01-08,92.63,93.47,91.99,92.68,10670000,92.68,92.713333


In [44]:
df['Variations'] = (df['Daily_average_price'] - df['GSPC.Close']) / df['GSPC.Close']
df.head()

,Index,GSPC.Open,GSPC.High,GSPC.Low,GSPC.Close,GSPC.Volume,GSPC.Adjusted,Daily_average_price,Variations
0,1970-01-02,92.06,93.54,91.79,93.00,8050000,93.00,92.776667,-0.002401
1,1970-01-05,93.00,94.25,92.53,93.46,11490000,93.46,93.413333,-0.000499
2,1970-01-06,93.46,93.81,92.13,92.82,11460000,92.82,92.920000,0.001077
3,1970-01-07,92.82,93.38,91.93,92.63,10010000,92.63,92.646667,0.000180
4,1970-01-08,92.63,93.47,91.99,92.68,10670000,92.68,92.713333,0.000360


In [45]:
def T_index(n_days=120, margin=0.025):
    T_list = []
    for i in df.index:
        T = 0
        Ci = df['GSPC.Close'].iloc[i]
        for j in range (i + 1, n_days + 1):
            variation = (df['Daily_average_price'].iloc[j] - Ci) / Ci
            if (variation > margin) or (variation < -margin) :
                T = T + variation
        T_list.append(T)
    return(T_list)

In [46]:
df['T_index'] = T_index()
df['T_index'].value_counts()

 0.000000    9908
-1.132061       2
-6.501097       2
-0.220324       2
-4.974173       1
             ... 
-5.045288       1
-4.057119       1
-4.201727       1
-4.273246       1
 0.087334       1
Name: T_index, Length: 112, dtype: int64

## Feature Selection

### Technical Indicators

**Average True Range(volatility):** A stock's range is the difference between the high and low prices on any given day. It reveals information about how volatile a stock is. Large ranges indicate high volatility and small ranges indicate low volatility. The range is measured the same way for options and commodities (high minus low) as they are for stocks. The average true range (ATR) is a simple moving average (SMA). Traders can use shorter or longer timeframes based on their trading preferences. Longer timeframes will be slower and will likely lead to fewer trading signals, while shorter timeframes will increase trading activity. 

**Stochastic Momentum Index(momentum):** The SMI is considered a refinement of the stochastic oscillator. It calculates the distance of the current closing price as it relates to the median of the high/low range of price. The SMI has a normal range of values between +100 and -100. When the present closing price is higher than the median, or midpoint value of the high/low range, the resulting value is positive. When the current closing price is lower than that of the midpoint of the high/low range, the SMI has a negative value. SMI is primarily used by traders or analysts to indicate overbought or oversold conditions in a market. It is used with volume indicators to show if the momentum carries significant selling or buying pressure. Traders also use the SMI as a general trend indicator, interpreting values above 40 as indicative of a bullish trend and negative values greater than -40 as showing a bearish trend.

**Welles Wilder's Directional Movement Index(trend):** Wilder’s DMI (ADX) consists of three indicators that measure a trend’s strength and direction. Wilder’s DMI, shows the strength of a trend, either up or down. ADX values help traders identify the strongest and most profitable trends to trade. The values are also important for distinguishing between trending and non-trending conditions. Many traders will use ADX readings above 25 to suggest that the trend is strong enough for trend-trading strategies.

**Aroon Indicator(trend):** The Aroon indicator is a technical indicator that is used to identify trend changes in the price of an asset, as well as the strength of that trend. In essence, the indicator measures the time between highs and the time between lows over a time period.  Particularly, the Aroon Oscillator uses aspects of the Aroon Indicator (Aroon Up and Aroon Down) to gauge the strength of a current trend and the likelihood that it will continue.

**Bollinger Bands(volatility):** Bollinger Bands are trend lines plotted above and below the SMA of the given stock at a specific standard deviation level. Bollinger Bands are great to observe the volatility of a given stock over a period of time. The volatility of a stock is observed to be lower when the space or distance between the upper and lower band is less. Similarly, when the space or distance between the upper and lower band is more, the stock has a higher level of volatility.  

**Chaikin Volatility(volatility):** The Chaikin Volatility Indicator is the difference between two moving averages of a volume weighted accumulation-distribution line. By comparing the spread between a security's high and low prices, it quantifies volatility as a widening of the range between the high and the low price. 

**Exponential Moving Average:** An exponential moving average (EMA) is a type of moving average (MA) that places a greater weight and significance on the most recent data points. The exponential moving average is also referred to as the exponentially weighted moving average. An exponentially weighted moving average reacts more significantly to recent price changes than a simple moving average (SMA), which applies an equal weight to all observations in the period. 

**Arm's Ease of Movement(Volume):** Richard Arms' Ease of Movement indicator is a technical study that attempts to quantify a mix of momentum and volume information into one value. The intent is to use this value to discern whether prices are able to rise, or fall, with little resistance in the directional movement. Theoretically, if prices move easily, they will continue to do so for a period of time that can be traded effectively.  

**MACD Oscillator:** Moving average convergence divergence (MACD) is a trend-following momentum indicator that shows the relationship between two moving averages of a security’s price. The MACD is calculated by subtracting the 26-period exponential moving average (EMA) from the 12-period EMA. 

**Money Flow Index:** The Money Flow Index is a volume indicator that helps in measuring the trading pressure on a particular asset. The MFI acts as an oscillator that ranges between 0 and 100. It helps us in signalling whether the stock is in the overbuying or overselling zone as well as divergences between the prices and this indicator helps in predicting a reversal in market trends. A stock is considered to be overbought if the MFI shows a fast price rise to a high level. Similarly, a stock is considered oversold if the MFI indicates a rapid decline in the price to a considerably low level. 

**Parabolic Stop-and-Reverse(trend):** The parabolic SAR indicator, developed by J. Wells Wilder, is used by traders to determine trend direction and potential reversals in price. The indicator uses a trailing stop and reverse method called "SAR," or stop and reverse, to identify suitable exit and entry points. The parabolic indicator generates buy or sell signals when the position of the dots moves from one side of the asset's price to the other.  

**Chande Momentum Oscillator(momentum):** The Chande momentum oscillator is a technical momentum indicator. The formula calculates the difference between the sum of recent gains and the sum of recent losses and then divides the result by the sum of all price movements over the same period. The Chande momentum oscillator is a technical indicator that uses momentum to identify relative strength or weakness in a market.

**Relative Strength Index(momentum:)** Relative Strength Index is a momentum indicator measuring and simplifying price movement. It identifies when stock is overbought or oversold. These gives entry and exit signals; but equally valuable, RSI serves as both a confirmation tool and as an early reversal signal that is confirmed by other technical signs.

In [47]:
# Average True Range indicator
atr = df.ta.atr(high='GSPC.High', low='GSPC.LOW', close='GSPC.Close', length=14, append=False)
df['ATR'] = atr

# Stochastic Momentum Index
smi = df.ta.smi(close='GSPC.Close', high='GSPC.High', low='GSPC.LOW', signal=14, append=False)
df['SMI'] = smi['SMI_5_20_14']

# Average Directional Momentum Index
adx = df.ta.adx(high='GSPC.High', low='GSPC.LOW', close='GSPC.Close', length=14, append=False)
df['ADX'] = adx['ADX_14']

# Aroon Indicator
aroon = df.ta.aroon(high='GSPC.High', low='GSPC.LOW', length=14, append=False)
df['aroon'] = aroon['AROONOSC_14']

# Bollinger Bands
bbands = df.ta.bbands(close='GSPC.Close', length=14, append=False)
df['Bbands'] = bbands['BBP_14_2.0']

# Chaikin Volatility
"""
Chaikin Volatility (CV)
Source: https://www.marketvolume.com/technicalanalysis/chaikinvolatility.asp
Params: 
    data: pandas DataFrame
    ema_periods: period for smoothing Highest High and Lowest Low difference
    change_periods: the period for calculating the difference between Highest High and Lowest Low
    high_col: the name of the HIGH values column
    low_col: the name of the LOW values column
    close_col: the name of the CLOSE values column
    
Returns:
    copy of 'data' DataFrame with 'chaikin_volatility' column added
"""
def chaikin_volatility(data, ema_periods=14, change_periods=10, high_col='<HIGH>', low_col='<LOW>', close_col='<CLOSE>'):
    data['ch_vol_hl'] = data[high_col] - data[low_col]
    data['ch_vol_ema'] = data['ch_vol_hl'].ewm(ignore_na=False, min_periods=0, com=ema_periods, adjust=True).mean()
    data['chaikin_volatility'] = 0.
    
    for index,row in data.iterrows():
        if index >= change_periods:
            
            prev_value = data.at[index-change_periods, 'ch_vol_ema']
            if prev_value == 0:
                #this is to avoid division by zero below
                prev_value = 0.0001
            data.loc[index,'chaikin_volatility'] = ((row['ch_vol_ema'] - prev_value)/prev_value)
            #data.set_value(index, 'chaikin_volatility', ((row['ch_vol_ema'] - prev_value)/prev_value))
            
    data = data.drop(['ch_vol_hl', 'ch_vol_ema'], axis=1)
        
    return data

chaikinVol = chaikin_volatility(df, high_col='GSPC.High', low_col='GSPC.Low', close_col='GSPC.Close')

## Exponential Moving Average
period_days = 14
def ema(prices, days, smoothing=2):
    ema = [sum(prices[:days]) / days]
    for price in prices[days:]:
        ema.append((price * (smoothing / (1 + days))) + ema[-1] * (1 - (smoothing / (1 + days))))
    return ema

ema = ema(df['GSPC.Close'], period_days) # Add this line to save EMA values in a list
fill_list = []
fill_list = [0] * (period_days-1)
ema = fill_list + ema
len(ema)
df['EMA'] = ema

# Arm's Ease of Movement Indicator
"""
Ease of Movement
Source: http://stockcharts.com/school/doku.php?id=chart_school:technical_indicators:ease_of_movement_emv
Params: 
    data: pandas DataFrame
    period: period for calculating EMV
    high_col: the name of the HIGH values column
    low_col: the name of the LOW values column
    vol_col: the name of the VOL values column
    
Returns:
    copy of 'data' DataFrame with 'emv' and 'emv_ema_[period]' columns added
"""
def ease_of_movement(data, period=14, high_col='<HIGH>', low_col='<LOW>', vol_col='<VOL>'):
    for index, row in data.iterrows():
        if index > 0:
            midpoint_move = (row[high_col] + row[low_col]) / 2 - (data.at[index - 1, high_col] + data.at[index - 1, low_col]) / 2
        else:
            midpoint_move = 0
        
        diff = row[high_col] - row[low_col]

        if diff == 0:
        #this is to avoid division by zero below
            diff = 0.000000001
            
        vol = row[vol_col]
        if vol == 0:
            vol = 1
        box_ratio = (vol / 100000000) / (diff)
        emv = midpoint_move / box_ratio
        data.loc[index,'emv'] = emv
        
    data['emv_ema_'+str(period)] = data['emv'].ewm(ignore_na=False, min_periods=0, com=period, adjust=True).mean()
 
    return data

ease_of_movement(df, high_col='GSPC.High', low_col='GSPC.Low', vol_col='GSPC.Volume')
df = df.drop(['emv_ema_14'], axis=1)  

# MACD Oscillator
macd = df.ta.macd(close='GSPC.Close', append=False)
df['MACD'] = macd['MACD_12_26_9']

# Money Flow Index
mfi= df.ta.mfi(high='GSPC.High', low='GSPC.Low', close='GSPC.Close', volume='GSPC.Volume', length=14, append=False)
#df['MACD'] = macd['MACD_12_26_9']
mfi.fillna(0)
df['MFI'] = mfi

# Parabolic Stop-and-Reverse Indicator
psar = df.ta.psar(high='GSPC.High', low='GSPC.LOW', close='GSPC.Close', append=False)
df['PSAR'] = psar['PSARl_0.02_0.2'].fillna(psar['PSARs_0.02_0.2'])

# Volatility Indicator
volat = np.sqrt(1 / 10 * pd.DataFrame.rolling(0.5 * np.log(df.loc[:, 'GSPC.High'] / df.loc[:, 'GSPC.Low']) ** 2 - (2 * np.log(2) - 1) * np.log(df.loc[:, 'GSPC.Close'] / df.loc[:, 'GSPC.Open']) ** 2, window=10).sum())
df['Volat'] = volat

# RunMean && RunSD
def runMean_SD(column_series, window_size):
    windows = column_series.rolling(window_size)
    # Get the window of series
    # of observations of specified window size
    windows = column_series.rolling(window_size)
  
    # Create a series of moving
    # averages of each window
    mean_moving_averages = windows.mean()
    sd_moving_averages = windows.std()  
    
    # Convert pandas series back to list
    mean_moving_averages_list = mean_moving_averages.tolist()
    sd_moving_averages_list = sd_moving_averages.tolist()
    
    # Remove null entries from the list
    mean_final_list = mean_moving_averages_list[window_size - 1:]
    sd_final_list = sd_moving_averages_list[window_size - 1:]
  
    return(mean_final_list, sd_final_list)
    
runMean, runSD = runMean_SD(df['GSPC.Close'], 10)
runSD.extend([0, 0, 0, 0, 0, 0, 0, 0, 0])
df['runSD'] = runSD
runMean.extend([0, 0, 0, 0, 0, 0, 0, 0, 0])
df['runMean'] = runMean

# Chande Momentum Oscillator
cmo = df.ta.cmo(close='GSPC.Close', append=False)
df['CMO'] = cmo.fillna(0)

# Relative Strength Index
rsi = df.ta.rsi(close='GSPC.Close', length=14, append=False)
df['RSI'] = rsi.fillna(0)

# Delt indicator
df['DELT'] = df['GSPC.Close'].pct_change(periods=14)

In [48]:
df = df.fillna(0)

In [49]:
df.isna().sum()

Index                  0
GSPC.Open              0
GSPC.High              0
GSPC.Low               0
GSPC.Close             0
GSPC.Volume            0
GSPC.Adjusted          0
Daily_average_price    0
Variations             0
T_index                0
ATR                    0
SMI                    0
ADX                    0
aroon                  0
Bbands                 0
ch_vol_hl              0
ch_vol_ema             0
chaikin_volatility     0
EMA                    0
emv                    0
MACD                   0
MFI                    0
PSAR                   0
Volat                  0
runSD                  0
runMean                0
CMO                    0
RSI                    0
DELT                   0
dtype: int64

## Feature Selection

### Split the data into 40% test and 60% training

In [50]:
X = df.drop(['T_index'], axis=1)
y = df['T_index']

In [56]:
X = X.drop(df.columns[0], axis=1)

In [57]:
X.columns

Index(['GSPC.Open', 'GSPC.High', 'GSPC.Low', 'GSPC.Close', 'GSPC.Volume',
       'GSPC.Adjusted', 'Daily_average_price', 'Variations', 'ATR', 'SMI',
       'ADX', 'aroon', 'Bbands', 'ch_vol_hl', 'ch_vol_ema',
       'chaikin_volatility', 'EMA', 'emv', 'MACD', 'MFI', 'PSAR', 'Volat',
       'runSD', 'runMean', 'CMO', 'RSI', 'DELT'],
      dtype='object')

In [74]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

In [75]:
sel = SelectFromModel(RandomForestRegressor(n_estimators = 100))
sel.fit(X_train, y_train)

sel.get_support()

array([False, False, False, False,  True, False, False, False,  True,
       False,  True, False, False, False, False, False,  True, False,
       False,  True, False, False, False, False, False,  True, False])

In [71]:
selected_feat= X_train.columns[(sel.get_support())]
print(selected_feat)

Index(['GSPC.Volume', 'ATR', 'ADX', 'EMA', 'MFI', 'RSI'], dtype='object')


## Technical Definitions

A **stop-loss order** is an order placed with a broker to buy or sell a security when it reaches a certain price.  Stop-loss orders are designed to limit an investor’s loss on a position in a security.  

A **swing low** is created when a low is lower than any other surrounding prices in a given period of time.